<a href="https://colab.research.google.com/github/DhasiM/cots-detection/blob/main/DSI_MasterNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project Goal And Description**

The aim of this project is to build a covolutional neural network model that helps to accurately identify starfish in real-time. The dataset used comes from underwater videos of the Australia's Great Barrier Reef which is inhabited by a huge variety of different species. In the project, the team applies several algorithms such as the YOLOv5 to train a model that is able to detect the target image of a start fish as accurately as possible.

# Exploratory Data Analysis

Here, the team tries to take a look at the features of the data. This is to have an overview of the input that we are using for the model and derive any possible modifications we may be able to apply to get better predictions.

In [1]:
from google.colab import drive #importing drive to access the data
drive.mount('/content/gdrive') # mounting my drive

Mounted at /content/gdrive


In [3]:
!ls gdrive/MyDrive/tensorflow-great-barrier-reef

example_sample_submission.csv  greatbarrierreef  train.csv
example_test.npy	       test.csv		 train_images


In [4]:
!ls gdrive/MyDrive/tensorflow-great-barrier-reef/train_images

video_0  video_1  video_2


In [5]:
image_folder = 'gdrive/MyDrive/tensorflow-great-barrier-reef/train_images/video_1'

In [7]:
# loading the data

import pandas as pd

DATA_PATH = 'gdrive/MyDrive/tensorflow-great-barrier-reef/'
train = pd.read_csv(DATA_PATH + 'train.csv')
train.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations
0,0,40258,0,0,0-0,[]
1,0,40258,1,1,0-1,[]
2,0,40258,2,2,0-2,[]
3,0,40258,3,3,0-3,[]
4,0,40258,4,4,0-4,[]


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23501 entries, 0 to 23500
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        23501 non-null  int64 
 1   sequence        23501 non-null  int64 
 2   video_frame     23501 non-null  int64 
 3   sequence_frame  23501 non-null  int64 
 4   image_id        23501 non-null  object
 5   annotations     23501 non-null  object
dtypes: int64(4), object(2)
memory usage: 1.1+ MB


In [9]:
(train[train['video_id']==0]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6708 entries, 0 to 6707
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        6708 non-null   int64 
 1   sequence        6708 non-null   int64 
 2   video_frame     6708 non-null   int64 
 3   sequence_frame  6708 non-null   int64 
 4   image_id        6708 non-null   object
 5   annotations     6708 non-null   object
dtypes: int64(4), object(2)
memory usage: 366.8+ KB


In [10]:
(train[train['video_id']==1]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8232 entries, 6708 to 14939
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   video_id        8232 non-null   int64 
 1   sequence        8232 non-null   int64 
 2   video_frame     8232 non-null   int64 
 3   sequence_frame  8232 non-null   int64 
 4   image_id        8232 non-null   object
 5   annotations     8232 non-null   object
dtypes: int64(4), object(2)
memory usage: 450.2+ KB


video_0 has 6708 images, while video_1 and video_2 has respectively 8232 and 8561 images.

In [11]:
train.duplicated().sum()

0

# Training Using YoloV5



In [ ]:
#import libraries
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
sys.path.append('./')
from joblib import Parallel, delayed
from IPython.display import display

# Detection

Items

# Validation

# Submission and Conclusion

*Items*